In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [2]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_yvle_zongyi=pd.DataFrame(columns=columns)
channel_yvle_zongyi

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [3]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [4]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [7]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[13]/div/a/span').click()  #选择频道

In [8]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[13]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'娱乐'

In [9]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[2]/a').click()  #选择子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[2]/a').text
sub_channel

'综艺'

In [10]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[2]/ul[2]/li[1]').click() #更换视频放置界面

In [11]:
print(datetime.datetime.now())

for month in range(1,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])

                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div[2]/div[3]/a'%video_loc).text #获取作者名
                release_date=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[1]/div[2]/div[3]/span').text
                
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名

                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_yvle_zongyi.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 
    


2020-12-15 23:00:31.101043


<ipython-input-11-faf26fe4a82f>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-11-faf26fe4a82f>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-11-faf26fe4a82f>:62: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-11-faf26fe4a82f>:185: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1WJ411E7co视频属性的抓取
BV号为1WJ411E7co的视频属性抓取结束


<ipython-input-11-faf26fe4a82f>:179: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1u7411q7q8视频属性的抓取
BV号为1u7411q7q8的视频属性抓取结束
开始BV号为1tJ411V7FK视频属性的抓取
BV号为1tJ411V7FK的视频属性抓取结束
开始BV号为1M7411v7x3视频属性的抓取
BV号为1M7411v7x3的视频属性抓取结束
开始BV号为1S7411e7jk视频属性的抓取
BV号为1S7411e7jk的视频属性抓取结束
开始BV号为1TJ411j7F6视频属性的抓取
BV号为1TJ411j7F6的视频属性抓取结束
开始BV号为1H7411q7bw视频属性的抓取
BV号为1H7411q7bw的视频属性抓取结束
开始BV号为1AJ411j76W视频属性的抓取
BV号为1AJ411j76W的视频属性抓取结束
开始BV号为1DJ41177Mq视频属性的抓取
BV号为1DJ41177Mq的视频属性抓取结束
开始BV号为1m7411v7zB视频属性的抓取
BV号为1m7411v7zB的视频属性抓取结束
开始BV号为1o7411i7PU视频属性的抓取
BV号为1o7411i7PU的视频属性抓取结束
开始BV号为1D7411z7gk视频属性的抓取
BV号为1D7411z7gk的视频属性抓取结束
开始BV号为1nJ41187dm视频属性的抓取
BV号为1nJ41187dm的视频属性抓取结束
开始BV号为1vJ411E7c1视频属性的抓取
开始BV号为1vJ411E7c1视频属性的抓取
开始BV号为1vJ411E7c1视频属性的抓取
开始BV号为1vJ411E7c1视频属性的抓取
BV号为1vJ411E7c1的视频属性抓取结束
开始BV号为1A7411k7hV视频属性的抓取
BV号为1A7411k7hV的视频属性抓取结束
开始BV号为1K7411q7fP视频属性的抓取
BV号为1K7411q7fP的视频属性抓取结束
开始BV号为1v7411e7Wk视频属性的抓取
BV号为1v7411e7Wk的视频属性抓取结束
开始BV号为1NJ411p779视频属性的抓取
BV号为1NJ411p779的视频属性抓取结束
开始BV号为1hJ411J7bw视频属性的抓取
BV号为1hJ411J7bw的视频属性抓取结束
开始BV号为1TJ41187Fd视频属性的抓取
BV号为1TJ41187Fd的视频属性抓取结束
开始BV号为1sJ411L77r

BV号为12741137YN的视频属性抓取结束
开始BV号为1F7411L7bR视频属性的抓取
BV号为1F7411L7bR的视频属性抓取结束
开始BV号为1h741137q8视频属性的抓取
BV号为1h741137q8的视频属性抓取结束
开始BV号为1YZ4y1j7s5视频属性的抓取
BV号为1YZ4y1j7s5的视频属性抓取结束
开始BV号为1uV411f7b3视频属性的抓取
BV号为1uV411f7b3的视频属性抓取结束
开始BV号为1dt4y1m7Fm视频属性的抓取
BV号为1dt4y1m7Fm的视频属性抓取结束
开始BV号为1cE411x7mf视频属性的抓取
BV号为1cE411x7mf的视频属性抓取结束
开始BV号为1Gz411b7jb视频属性的抓取
BV号为1Gz411b7jb的视频属性抓取结束
开始BV号为1SE411F7LN视频属性的抓取
开始BV号为1SE411F7LN视频属性的抓取
BV号为1SE411F7LN的视频属性抓取结束
开始BV号为1ME411n7eW视频属性的抓取
BV号为1ME411n7eW的视频属性抓取结束
开始BV号为1bE411N7cy视频属性的抓取
BV号为1bE411N7cy的视频属性抓取结束
开始BV号为1dE411J7np视频属性的抓取
BV号为1dE411J7np的视频属性抓取结束
开始BV号为1TE411V77q视频属性的抓取
BV号为1TE411V77q的视频属性抓取结束
开始BV号为1Pz411b71x视频属性的抓取
BV号为1Pz411b71x的视频属性抓取结束
开始BV号为1h7411d73q视频属性的抓取
BV号为1h7411d73q的视频属性抓取结束
开始BV号为1AE411W7Ur视频属性的抓取
BV号为1AE411W7Ur的视频属性抓取结束
开始BV号为1XA41187qH视频属性的抓取
BV号为1XA41187qH的视频属性抓取结束
开始BV号为1y7411f7LK视频属性的抓取
BV号为1y7411f7LK的视频属性抓取结束
开始BV号为18E411N79f视频属性的抓取
BV号为18E411N79f的视频属性抓取结束
开始BV号为1AE411c7Ls视频属性的抓取
BV号为1AE411c7Ls的视频属性抓取结束
开始BV号为1y741197NV视频属性的抓取
BV号为1y741197NV的视

BV号为1Ua4y1i7xP的视频属性抓取结束
开始BV号为1Dt4y117bT视频属性的抓取
BV号为1Dt4y117bT的视频属性抓取结束
开始BV号为1p541147Qr视频属性的抓取
BV号为1p541147Qr的视频属性抓取结束
开始BV号为1Gk4y167Ex视频属性的抓取
BV号为1Gk4y167Ex的视频属性抓取结束
开始BV号为1LQ4y1P7Wg视频属性的抓取
开始BV号为1LQ4y1P7Wg视频属性的抓取
BV号为1LQ4y1P7Wg的视频属性抓取结束
开始BV号为1Np4y1X7YK视频属性的抓取
BV号为1Np4y1X7YK的视频属性抓取结束
开始BV号为18k4y167LA视频属性的抓取
BV号为18k4y167LA的视频属性抓取结束
开始BV号为1WK4y187Sg视频属性的抓取
BV号为1WK4y187Sg的视频属性抓取结束
开始BV号为1Rz411v7za视频属性的抓取
BV号为1Rz411v7za的视频属性抓取结束
开始BV号为14f4y1U7tm视频属性的抓取
BV号为14f4y1U7tm的视频属性抓取结束
开始BV号为1Cz411v7fb视频属性的抓取
BV号为1Cz411v7fb的视频属性抓取结束
开始BV号为11Q4y1A7A2视频属性的抓取
BV号为11Q4y1A7A2的视频属性抓取结束
开始BV号为1VZ4y1W7G5视频属性的抓取
BV号为1VZ4y1W7G5的视频属性抓取结束
开始BV号为1Et4y117Ve视频属性的抓取
BV号为1Et4y117Ve的视频属性抓取结束
开始BV号为1GQ4y1N7nV视频属性的抓取
BV号为1GQ4y1N7nV的视频属性抓取结束
开始BV号为18K4y1t7ob视频属性的抓取
BV号为18K4y1t7ob的视频属性抓取结束
开始BV号为1j54y1X7hm视频属性的抓取
BV号为1j54y1X7hm的视频属性抓取结束
开始BV号为11K41157EG视频属性的抓取
BV号为11K41157EG的视频属性抓取结束
开始BV号为1pZ4y1W7MA视频属性的抓取
BV号为1pZ4y1W7MA的视频属性抓取结束
开始BV号为1Tk4y1k7iw视频属性的抓取
BV号为1Tk4y1k7iw的视频属性抓取结束
开始BV号为1SC4y1W73m视频属性的抓取
BV号为1SC4y1W73m的视

<ipython-input-11-faf26fe4a82f>:188: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1Ra4y1a7KQ视频属性的抓取
BV号为1Ra4y1a7KQ的视频属性抓取结束
开始BV号为1kz4y1X7Jc视频属性的抓取
BV号为1kz4y1X7Jc的视频属性抓取结束
开始BV号为19A411e7Ay视频属性的抓取
BV号为19A411e7Ay的视频属性抓取结束
开始BV号为1bz4y1X7Wf视频属性的抓取
BV号为1bz4y1X7Wf的视频属性抓取结束
开始BV号为17Z4y1u7xc视频属性的抓取
BV号为17Z4y1u7xc的视频属性抓取结束
开始BV号为1ng4y1i7Wu视频属性的抓取
BV号为1ng4y1i7Wu的视频属性抓取结束
开始BV号为1sz411v7Ch视频属性的抓取
BV号为1sz411v7Ch的视频属性抓取结束
开始BV号为1j54y1S7tp视频属性的抓取
BV号为1j54y1S7tp的视频属性抓取结束
开始BV号为1hK411H7Wo视频属性的抓取
BV号为1hK411H7Wo的视频属性抓取结束
开始BV号为1Yt4y1X7sx视频属性的抓取
BV号为1Yt4y1X7sx的视频属性抓取结束
开始BV号为1nA411e7Ba视频属性的抓取
BV号为1nA411e7Ba的视频属性抓取结束
开始BV号为1w5411a7Ko视频属性的抓取
BV号为1w5411a7Ko的视频属性抓取结束
开始BV号为1u54y1q79i视频属性的抓取
BV号为1u54y1q79i的视频属性抓取结束
开始BV号为1ja4y1e75d视频属性的抓取
BV号为1ja4y1e75d的视频属性抓取结束
开始BV号为1zD4y1Q7oT视频属性的抓取
BV号为1zD4y1Q7oT的视频属性抓取结束
开始BV号为1Mp4y1q7xJ视频属性的抓取
BV号为1Mp4y1q7xJ的视频属性抓取结束
开始BV号为1UV41167UP视频属性的抓取
BV号为1UV41167UP的视频属性抓取结束
开始BV号为1bT4y1E7jz视频属性的抓取
开始BV号为1bT4y1E7jz视频属性的抓取
BV号为1bT4y1E7jz的视频属性抓取结束
开始BV号为1ZC4y187Ez视频属性的抓取
BV号为1ZC4y187Ez的视频属性抓取结束
开始BV号为1Rt4y197hp视频属性的抓取
BV号为1Rt4y197hp的视频属性抓取结束
开始BV号为1qv411B7SH

开始BV号为1Mf4y1X7oJ视频属性的抓取
BV号为1Mf4y1X7oJ的视频属性抓取结束
开始BV号为1Gh41197HM视频属性的抓取
BV号为1Gh41197HM的视频属性抓取结束
开始BV号为1F54y117mt视频属性的抓取
BV号为1F54y117mt的视频属性抓取结束
开始BV号为1Vt4y1S7Pm视频属性的抓取
BV号为1Vt4y1S7Pm的视频属性抓取结束
开始BV号为1yf4y1D77D视频属性的抓取
BV号为1yf4y1D77D的视频属性抓取结束
开始BV号为1BK4y187Qn视频属性的抓取
BV号为1BK4y187Qn的视频属性抓取结束
开始BV号为1Wk4y117Q2视频属性的抓取
BV号为1Wk4y117Q2的视频属性抓取结束
开始BV号为1RA411E7xy视频属性的抓取
BV号为1RA411E7xy的视频属性抓取结束
开始BV号为1ZK411K79M视频属性的抓取
BV号为1ZK411K79M的视频属性抓取结束
开始BV号为1X5411j7Qz视频属性的抓取
BV号为1X5411j7Qz的视频属性抓取结束
开始BV号为1UZ4y1N7ZF视频属性的抓取
BV号为1UZ4y1N7ZF的视频属性抓取结束
开始BV号为1H5411b77S视频属性的抓取
BV号为1H5411b77S的视频属性抓取结束
开始BV号为1Xp4y1Y7Ey视频属性的抓取
BV号为1Xp4y1Y7Ey的视频属性抓取结束
开始BV号为1154y1C7pd视频属性的抓取
BV号为1154y1C7pd的视频属性抓取结束
开始BV号为1bz4y1Z7Wd视频属性的抓取
BV号为1bz4y1Z7Wd的视频属性抓取结束
开始BV号为1sV41117wj视频属性的抓取
BV号为1sV41117wj的视频属性抓取结束
开始BV号为1Ki4y1E7JV视频属性的抓取
BV号为1Ki4y1E7JV的视频属性抓取结束
开始BV号为1w5411j7qc视频属性的抓取
BV号为1w5411j7qc的视频属性抓取结束
开始BV号为1vZ4y1N7fk视频属性的抓取
BV号为1vZ4y1N7fk的视频属性抓取结束
开始BV号为1Qy4y1k7Qz视频属性的抓取
BV号为1Qy4y1k7Qz的视频属性抓取结束
开始BV号为1pt4y1q7Vc视频属性的抓取
BV号为1pt4y1q7Vc的视

In [28]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-28-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [12]:
channel_yvle_zongyi.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70
1月,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80
2月,91,91,91,91,91,91,91,91,91,91,91,91,91,91,91,91,91,91,91,91
3月,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68
4月,86,86,86,86,86,86,86,86,86,86,86,86,86,86,86,86,86,86,86,86
5月,81,81,81,81,81,81,81,81,81,81,81,81,81,81,81,81,81,81,81,81
6月,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82
7月,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73
8月,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67


In [15]:
channel_yvle_zongyi.to_json('channel_yvle_zongyi.json')

In [13]:
pd.set_option('display.max_rows', None)

In [14]:
channel_yvle_zongyi[channel_yvle_zongyi['时间']=='1月']

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
1WJ411E7co,1月,2020-01-01 00:22:03,开心嘴炮,个人认证,193.2万,252,精品抄成垃圾！怒火吐槽大型女德节目《你怎么这么好看》,娱乐,综艺,普通视频,...,"[吴昕, 综艺, 搞笑, 娱乐, 综艺, 明星, 粉雄救兵, 吐槽, 你怎么这么好看, 真人...",12:22,730.1万,55.3万,38.7万,10.4万,8.2万,32100,14.0万,否
1u7411q7q8,1月,2020-01-01 00:22:03,哔哩哔哩弹幕网,机构认证,305.5万,91,BILIBILI POWER UP 2019年度UP主颁奖全程回顾,娱乐,综艺,普通视频,...,"[综艺, 娱乐, 综艺, 年度UP主颁奖, BILIBILI POWER UP, 百大UP主...",105:08,569.6万,16.1万,6.5万,9.9万,1.4万,9838,45.9万,是
1tJ411V7FK,1月,2020-01-01 00:22:03,哔哩哔哩UP主执事,机构认证,116.6万,146,【年度UP主颁奖】有内味吗？2019年度百大UP主获（xin）奖（nian）感（zhu）言（fu）,娱乐,综艺,普通视频,...,"[综艺, 娱乐, 综艺, 十周年成就奖, BILIBILI POWER UP, POWER ...",30:45,458.2万,13.9万,3.4万,3.9万,8554,10977,23.9万,否
1M7411v7x3,1月,2020-01-01 00:22:03,枫思love摄恋,无,3.4万,136,【周深】《快本》出场表演“左手指月”高音震惊全场,娱乐,综艺,普通视频,...,"[周深, 国内综艺, 综艺, 娱乐, 综艺, 左手指月]",02:38,402.3万,8.3万,1.1万,5.4万,1.8万,4591,2.7万,是
1S7411e7jk,1月,2020-01-01 00:22:03,暴走漫画,个人认证,584.5万,177,【暴走大事件】新春篇：暴走大富翁，亿元难题首遇挑战,娱乐,综艺,普通视频,...,"[综艺, 搞笑, 娱乐, 综艺, 超大事件, 王尼玛, 搞笑视频, 真人秀]",14:12,328.6万,32.0万,37.5万,12.3万,4.1万,17197,11.4万,是
1TJ411j7F6,1月,2020-01-01 00:22:03,腾讯综艺,机构认证,57.4万,989,【我们的歌】周深李克勤翻唱《神雕侠侣》主题曲《天下有情人》,娱乐,综艺,普通视频,...,"[我们的歌, 李克勤, 周深, 综艺, 娱乐, 综艺, 天下有情人]",04:39,243.2万,7.6万,3.9万,6.2万,2.3万,4900,1.9万,否
1H7411q7bw,1月,2020-01-01 00:22:03,哔哩哔哩UP主执事,机构认证,116.6万,146,【年度UP主颁奖】全程高燃，属于2019年度百大UP主的高光时刻,娱乐,综艺,普通视频,...,"[综艺, 娱乐, 综艺, BILIBILI POWER UP, 年度UP主颁奖, POWER...",18:14,204.7万,4.3万,8525,2.7万,2662,1584,8.7万,是
1AJ411j76W,1月,2020-01-01 00:22:03,大宝剑联盟,个人认证,100.9万,8723,【日综】日本16岁的男高中生井手上漠，因为比女生还可爱的长相引发了超高关注，小哥哥长得好看就...,娱乐,综艺,普通视频,...,"[日本, 综艺, 搞笑, 娱乐, 综艺, 娱乐, youtube]",01:25,204.3万,4.9万,4548,2.9万,3.0万,3186,3155,否
1DJ41177Mq,1月,2020-01-01 00:22:03,话很多的小姐姐们,个人认证,154.8万,441,【小姐姐们】垃圾节目暴露中国女性处境，窒息吐槽毁三观抄袭综艺《你怎么这么好看》！,娱乐,综艺,活动作品,...,"[吴昕, 综艺, 娱乐, 综艺, 明星, 粉雄救兵, 你怎么这么好看, 昆凌, 素人改造, ...",11:42,199.3万,8.4万,2.6万,9090,1.2万,9470,4.4万,否
1m7411v7zB,1月,2020-01-01 00:22:03,笑兮清悦,无,1666,154,《吐槽大会》王建国极限一换三，爆笑脱口秀,娱乐,综艺,普通视频,...,"[李诞, 脱口秀, 综艺, 娱乐, 综艺, 王建国, 张绍刚, 《吐槽大会》]",08:16,184.0万,3.3万,2681,1.2万,4449,697,3775,否


In [27]:
channel_youxi_wangluo

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
1MJ411H77i,1月,2020-01-14 15:10:24,原神,机构认证,428.3万,53,《原神》新地区预告-「浮世浮生千岩间」,游戏,网络游戏,普通视频,...,"[原神, 游戏, 网络游戏, 手机游戏, miHoYo, 香菱, 派蒙, 米哈游, PC, 璃月]",01:12,526.3万,5.1万,1.9万,1.1万,1.8万,11442,5698,是
1oJ411V7s1,1月,2020-01-14 15:10:24,丶小禾,无,1.1万,222,当火线妹关闭直播美颜功能以后，才知道她做主播有多幸苦！,游戏,网络游戏,普通视频,...,"[绝地求生, 吃鸡, 游戏, 网络游戏, 网络游戏, 游戏, 火线妹]",01:12,384.1万,3.7万,1164,6585,538,2051,2506,否
